In [3]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
#unzip Autopilot folder as data is inside there
#!cp  /content/drive/Hackthon/Innoplexus/* .
#!cp new_html.csv  /content/drive/Hackthon/Innoplexus/

In [6]:
!unzip train.zip

Archive:  train.zip
  inflating: train.csv               
  inflating: html_data.csv           


In [1]:
!ls

datalab			Innoplexus_train_01.ipynb
drive			Innoplexus_train_02.ipynb
glove.6B.100d.txt	LSTM_Innoplexus_train_03.ipynb
glove.6B.200d.txt	new_html.csv
glove.6B.300d.txt	sample_submission_poy1UIu.csv
glove.6B.50d.txt	test_nvPHrOx.csv
glove.6B.zip		test_predicted.csv
gru_100_ep_30_model.h5	train.csv
html_data.csv		train.zip


In [2]:
import pandas as pd
import numpy as np
import re
from keras.models import Sequential
from keras.layers import Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input,  Flatten
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D, Embedding

from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix

from numpy.random import RandomState
from sklearn.preprocessing import LabelEncoder
from keras.layers import LSTM,GRU
SEED = RandomState(1234567890)

Using TensorFlow backend.


In [0]:
Data = pd.read_csv("train.csv")

In [4]:
Data.shape

(53447, 4)

In [5]:
Data.head()

,Webpage_id,Domain,Url,Tag
0,1,www.fiercepharma.com,http://www.fiercepharma.com/marketing/tecfider...,news
1,2,www.fiercepharma.com,http://www.fiercepharma.com/pharma/novo-equipp...,news
2,3,www.fiercepharma.com,http://www.fiercepharma.com/pharma/another-exe...,news
3,4,www.fiercepharma.com,http://www.fiercepharma.com/pharma/teva-buy-bi...,news
4,5,www.fiercepharma.com,http://www.fiercepharma.com/marketing/actress-...,news


In [0]:
def url_str_conv(s):
  #ss =re.sub('http://\w.*/','',s)
  #ss =re.sub('https://\w.*/','',s)
  i1 = s.find("//")
  i2 = s.find("/",i1+2)
  if i1==-1 or i2 == -1:
    i3 = s.find(".com")
    ss = s[i1+2:i3]
    ss =re.sub('www.','',ss)
    #print(ss)
    ss =re.findall(r'\w\w+',ss)
    ss = ' '.join(ss)
    return ss
  ss = s[i2+1:]
  ss =re.sub('www.','',ss)
  ss =re.findall(r'\w\w+',ss)
  ss = ' '.join(ss)
  return ss

In [7]:
q= 'https://www.clinicaltrialsregister.eu/ctr-search/trial/2006-006214-16/GB'
url_str_conv(q)

'ctr search trial 2006 006214 16 GB'

In [8]:

Data['new_url'] = list(map(url_str_conv,Data.Url))
Data.head()

,Webpage_id,Domain,Url,Tag,new_url
0,1,www.fiercepharma.com,http://www.fiercepharma.com/marketing/tecfider...,news,marketing tecfidera gilenya and aubagio way ba...
1,2,www.fiercepharma.com,http://www.fiercepharma.com/pharma/novo-equipp...,news,pharma novo equipped to weather storm diabetes...
2,3,www.fiercepharma.com,http://www.fiercepharma.com/pharma/another-exe...,news,pharma another exec departs troubled endo and ...
3,4,www.fiercepharma.com,http://www.fiercepharma.com/pharma/teva-buy-bi...,news,pharma teva buy biosim specialist celltrion it...
4,5,www.fiercepharma.com,http://www.fiercepharma.com/marketing/actress-...,news,marketing actress marissa tomei partners aller...


In [0]:
Data.sort_values('Tag',inplace=True)

In [0]:
MAX_SEQUENCE_LENGTH = 500
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
GLOVE_FILE = 'glove.6B.100d.txt'

In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
def tokenize_data(X_raw, Y_raw):
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS,char_level=True)
    tokenizer.fit_on_texts(X_raw)
    sequences = tokenizer.texts_to_sequences(X_raw)
    word_index = tokenizer.word_index
    X_processed = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    label_encoder_y = LabelEncoder()
    Y_processed=to_categorical(label_encoder_y.fit_transform(Y_raw))

    return X_processed, Y_processed, word_index,tokenizer,label_encoder_y

In [0]:
def tokenize_test_data(X_raw, tokenizer):
    sequences = tokenizer.texts_to_sequences(X_raw)
    word_index = tokenizer.word_index
    X_processed = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

    return X_processed,word_index

In [0]:
def split_the_data(X_processed, Y_processed):
  count=1
  #X=list(X_processed)
  #Y=list(Y_processed)
  x_train=[]
  y_train=[]
  x_val=[]
  y_val=[]
  for i,j in zip(X_processed,Y_processed):
    if count%3==0:
      x_val.append(i)
      y_val.append(j)
    else:
      x_train.append(i)
      y_train.append(j)
    count+=1
  x_train=np.asarray(x_train)
  x_val=np.asarray(x_val)
  y_train=np.asarray(y_train)
  y_val=np.asarray(y_val)
  return x_train, x_val, y_train, y_val

In [0]:
def make_embedding_layer(word_index):
    embeddings = get_embeddings()
    nb_words = min(MAX_NB_WORDS, len(word_index))
    embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))

    for word, i in word_index.items():
        if i >= MAX_NB_WORDS:
            continue
        embedding_vector = embeddings.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    embedding_layer = Embedding(nb_words, EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False)
    return embedding_layer

In [0]:
def get_embeddings():
    embeddings = {}
    with open(GLOVE_FILE, 'r') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings[word] = coefs
    return embeddings

In [0]:
X_processed, Y_processed, word_index,tokenizer,label_encoder_y = tokenize_data(Data.new_url,Data.Tag)
x_train, x_val, y_train, y_val = split_the_data(X_processed, Y_processed)
#embedded_sequences = make_embedding_layer(word_index)
nb_words = min(MAX_NB_WORDS, len(word_index))

In [0]:
def make_model_lstm(labels_index):
    #embedded_sequences = make_embedding_layer(word_index)
    model = Sequential([
        Embedding(nb_words, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH),
        LSTM(200),
        #LSTM(100),
        #Flatten(),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(28, activation='relu'),
        Dense(labels_index, activation='softmax')
    ])

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

    return model

In [0]:
def make_model_gru(labels_index):
    #embedded_sequences = make_embedding_layer(word_index)
    model = Sequential([
        embedded_sequences,
        GRU(100),
        #GRU(50),
        #Flatten(),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(labels_index, activation='softmax')
    ])

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

    return model

In [0]:
labels_index=9
#model = make_model_gru(labels_index, embedded_sequences)
model = make_model_lstm(labels_index)

In [0]:
model.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=10, batch_size=256,)
#model.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=30, batch_size=256,)
#model.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=30, batch_size=64,)

In [0]:
model.save("gru_100_ep_30_model.h5")
!cp gru_100_ep_30_model.h5  /content/drive/Hackthon/Innoplexus/

In [0]:
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

In [0]:
s = [np.argmax(y, axis=None, out=None) for y in y_val]

In [0]:
#pre_train = model.predict_classes(x_train)
pre_val = model.predict_classes(x_val)

In [37]:
#print(f1_score(s,pre_val,average='weighted'))
print(confusion_matrix(s,pre_val,))

[[ 940    0    0    0    0    6    0    0    0]
 [   2 1303    0    0   33  182    3   30    2]
 [   1    2 1412    0   34   46    3    3    0]
 [   0    7    0  320   27   46    1   42    0]
 [   4   27   39    8 1880  628   31   43    4]
 [   3   50   48    4  153 5340  160   43    5]
 [   0    1    0    0    3  184 1542    2    0]
 [   3   11    0   24   44  101    4 2380    1]
 [   0    0    0    0    0    0    0    0  600]]


In [65]:
print(pre_val[:5])
print(s[:5])

[0 0 0 0 0]
['clinicalTrials' 'publication' 'publication' 'publication' 'publication']
[[ 0  0  0 ... 27 25 15]
 [ 0  0  0 ...  1 13  6]
 [ 0  0  0 ... 19 28 25]
 ...
 [ 0  0  0 ... 51 29 30]
 [ 0  0  0 ...  6 32 10]
 [ 0  0  0 ... 16 28  7]]


In [0]:
Data_test = pd.read_csv("test_nvPHrOx.csv")

In [39]:
Data_test.shape

(25787, 3)

In [40]:
Data_test.head()

,Webpage_id,Domain,Url
0,31,isrctn.com,http://www.isrctn.com/ISRCTN57801413
1,32,www.clinicaltrialsregister.eu,https://www.clinicaltrialsregister.eu/ctr-sear...
2,33,www.clinicaltrialsregister.eu,https://www.clinicaltrialsregister.eu/ctr-sear...
3,34,www.clinicaltrialsregister.eu,https://www.clinicaltrialsregister.eu/ctr-sear...
4,35,www.clinicaltrialsregister.eu,https://www.clinicaltrialsregister.eu/ctr-sear...


In [0]:

Data_test['new_url'] = list(map(url_str_conv,Data_test.Url))


In [42]:
Data_test.Url[1]

'https://www.clinicaltrialsregister.eu/ctr-search/trial/2006-006214-16/GB'

In [0]:
x_test,word_index = tokenize_test_data(Data_test['new_url'], tokenizer)

In [51]:
len(x_test)

25787

In [53]:
y_test = model.predict_classes(x_test)
y_test.shape

(25787,)

In [54]:
s =label_encoder_y.inverse_transform(y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [55]:
print(len(s))
Data_test.shape

25787


(25787, 4)

In [0]:
Data_test["Tag"] = s

In [0]:
Data_test.to_csv("test_predicted.csv",index=False,columns=['Webpage_id',"Tag"])

In [0]:
from google.colab import files
files.download("test_predicted.csv")

In [68]:
Data_test.tail(20)

,Webpage_id,Domain,Url,new_url,Tag
25767,79247,www.broadinstitute.org,https://www.broadinstitute.org/bios/orit-rozen...,bios orit rozenblatt rosen,profile
25768,79252,www.broadinstitute.org,https://www.broadinstitute.org/bios/paola-arlotta,bios paola arlotta,profile
25769,79253,www.broadinstitute.org,https://www.broadinstitute.org/bios/pardis-sabeti,bios pardis sabeti,profile
25770,79254,www.broadinstitute.org,https://www.broadinstitute.org/bios/phillip-sharp,bios phillip sharp,profile
25771,79255,www.broadinstitute.org,https://www.broadinstitute.org/bios/jen-pan,bios jen pan,profile
25772,79256,www.broadinstitute.org,https://www.broadinstitute.org/bios/paul-blainey,bios paul blainey,profile
25773,79257,www.broadinstitute.org,https://www.broadinstitute.org/bios/partha-nag,bios partha nag,profile
25774,79258,www.broadinstitute.org,https://www.broadinstitute.org/bios/roy-perlis,bios roy perlis,profile
25775,79259,www.broadinstitute.org,https://www.broadinstitute.org/bios/sara-garam...,bios sara garamszegi,profile
25776,79260,www.broadinstitute.org,https://www.broadinstitute.org/bios/scott-younger,bios scott younger,profile
